In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from gensim.models import Word2Vec
import numpy as np

import random
from ast import literal_eval
from torch.utils.data import Dataset,DataLoader
import itertools
from pprint import pprint
from tqdm.notebook import tqdm


In [2]:
import wandb
wandb.login(key='7eb3203fd8e9f410dc5b609cccd4054725ccdc96')
run = wandb.init(project="Swapno_Rec", job_type="train")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/shadman/.netrc
wandb: Currently logged in as: sknahin (cloth_transfer). Use `wandb login --relogin` to force relogin


In [3]:
if torch.cuda.is_available():
    
    device=torch.device("cuda:0")
    print("Running on the GPU")
    torch.cuda.empty_cache()
  #dataType=torch.float32
else:
    device=torch.device("cpu")
    print("Running on the CPU")
  #dataType=torch.float32

Running on the CPU


In [4]:
import json
def save_json(data,name):
    with open(name+'.json', 'w',encoding='utf8') as f:
        json.dump(data, f,ensure_ascii=False)
        
def save_jsonl(data,name):
    with open(name+'.jsonl', 'w') as f:
        for entry in data:
            json.dump(entry, f)
            outfile.write('\n')

def load_json(path):
    f = open (path, "r")
    data = json.loads(f.read())
    f.close()
    return data

def load_jsonl(path):
    f = open(path,"r").readlines()
    data=[]
    for i in f:
        d = json.loads(i)
        data.append(d)
    return data

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
vect_model = Word2Vec.load("/kaggle/input/swapno-rec-data/global_embed10_25/global_embed10_25.model")

products = pd.read_csv("/kaggle/input/shwapno-small/products_small.csv")
products.columns = products.iloc[0].to_list()
products = products.drop([0])
products = products.reset_index(drop=True)
products["ProductCode"] = products["ProductCode"].apply(lambda x: "P-"+str(x))

customers = pd.read_csv("/kaggle/input/shwapno-small/customer_small.csv")
customers["CustomerCode"] = customers["CustomerCode"].apply(lambda x: "C-"+str(x))

sorted_csv = pd.read_csv("/kaggle/input/swapno-rec-data/new_sorted_basket.csv")
sorted_csv["InvoiceDate"]=sorted_csv["InvoiceDate"].apply(lambda x: x[2:-2])


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/swapno-rec-data/global_embed10_25/global_embed10_25.model'

In [ ]:
global_key2idx = vect_model.wv.key_to_index
global_key2idx.update({"<s>":43620,"</s>":43621,"<v>":43622,"<pad>":43623})
global_idx2key=vect_model.wv.index_to_key
global_idx2key.extend(["<s>","</s>","<v>","<pad>"])
save_json(global_idx2key,"global_idx2key")
save_json(global_key2idx,"global_key2idx")

all_prods = list(products["ProductCode"])
all_per = list(customers["CustomerCode"])
all_dates = list(set(list(sorted_csv["InvoiceDate"])))


global_code2name = dict(zip(list(products["ProductCode"]),list(products["ProductName"])))
customer_code2name = dict(zip(list(customers["CustomerCode"]),list(customers["CustomerName"])))
global_code2name.update(customer_code2name)
global_code2name.update(dict(zip(all_dates,all_dates)))
global_code2name.update({"<s>":"start","</s>":"end","<v>":"void","<pad>":"pad"})


In [ ]:
sorted_csv_2 = sorted_csv.copy()
sorted_csv_2["CustomerCode"]=sorted_csv_2["CustomerCode"].apply(lambda x: literal_eval(x)[0])
sorted_csv_2["basket_sorted"]=sorted_csv_2["basket_sorted"].apply(lambda x: literal_eval(x))

sorted_csv_3 = sorted_csv_2.sort_values("InvoiceDate").groupby("CustomerCode").agg(list).reset_index()
sorted_csv_3 = sorted_csv_3.drop(columns=["InvoiceNo","ProductCode","InvoiceDate","basket"])
sorted_csv_3.to_csv("train_data.csv", index = False)
sorted_csv_3.head()

In [ ]:
def sort_by_priority_list(values, priority):
    priority_dict = dict(
        zip(
            priority,
            range(len(priority)),
        ),
    )
    priority_getter = priority_dict.get # dict.get(key)
    return sorted(values, key=priority_getter)

def collate_batch(batch):
    prev_prods = [i["prev_prods"] for i in batch]
    next_basket = [i["next_basket"] for i in batch]
    src_mask = [i["src_mask"] for i in batch]
    tgt_mask = [i["tgt_mask"] for i in batch]
    
    batch = {}
    batch["prev_prods"] = torch.tensor(list(zip(*itertools.zip_longest(*prev_prods, fillvalue=43623))))
    batch["next_basket"] = torch.tensor(list(zip(*itertools.zip_longest(*next_basket, fillvalue=43623))))
    batch["src_mask"] = torch.tensor(list(zip(*itertools.zip_longest(*src_mask, fillvalue=0))),dtype = torch.float32)
    batch["tgt_mask"] = torch.tensor(list(zip(*itertools.zip_longest(*tgt_mask, fillvalue=0))),dtype = torch.float32)
    
    return batch

In [ ]:

class Swapno_Data(Dataset):
    def __init__(self,train=True):
        super().__init__()
        self.train=train
        self.data_df = pd.read_csv("/kaggle/working/train_data.csv")
        self.data_df["basket_sorted"]=self.data_df["basket_sorted"].apply(lambda x: literal_eval(x))
        self.key2idx = load_json("/kaggle/working/global_key2idx.json")
        self.priority_dict = load_json("/kaggle/input/swapno-rec-data/priority_dict.json")
        self.max_prev_basket = 5
        
    def __getitem__(self,idx):
        
        customer_no = idx
        customer_id = self.data_df.iloc[customer_no]["CustomerCode"]
        
        pur_bas = self.data_df.iloc[customer_no]["basket_sorted"]

        total_pur = len(pur_bas)
        take_one_pur = random.randint(0,total_pur-1)
        
        if take_one_pur==0:
            prev_basket = ["<v>","</s>"]
            curr_basket = pur_bas[take_one_pur]+["</s>"]
        else:
            prev_basket = []
            
            for tp in range(max([0,take_one_pur-self.max_prev_basket]),take_one_pur):
                prev_basket.extend(pur_bas[tp][2:])

            prev_basket = sort_by_priority_list(list(set(prev_basket)), self.priority_dict[customer_id])+["</s>"]
            curr_basket = pur_bas[take_one_pur]+["</s>"]
            
        model_input = {}
        model_input["prev_prods"] = [self.key2idx[i] for i in prev_basket] 
        model_input["next_basket"] = [self.key2idx[i] for i in curr_basket] 
        model_input["src_mask"] = [1]*len(prev_basket) 
        model_input["tgt_mask"] = [1]*len(curr_basket)
        
        return model_input
    
    def __len__(self):
        return len(self.data_df)
    

In [ ]:
train_dataset = Swapno_Data(train=True)
train_loader = DataLoader(train_dataset, collate_fn=collate_batch, batch_size=10, shuffle=True, pin_memory=True, drop_last=True, num_workers=2)

check_loader = DataLoader(train_dataset, collate_fn=collate_batch, batch_size=10, shuffle=True, pin_memory=True, drop_last=True, num_workers=2)

In [ ]:
for i in train_loader:
    print(i)
    break

In [ ]:
class EncoderDecoderTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers):
        super(EncoderDecoderTransformer, self).__init__()
        
        # Encoder
        self.global_embedding = nn.Embedding(vocab_size, d_model)
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, batch_first=True),num_encoder_layers)
        
        # Decoder
        self.decoder = nn.TransformerDecoder(nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, batch_first=True),num_decoder_layers)

        self.output_linear = nn.Linear(d_model, vocab_size)
        self.soft = nn.Softmax(dim=-1)
    
    def forward(self, src,src_key_padding_mask,tgt,tgt_key_padding_mask):
        tgt_len = tgt.shape[1]
        
#         print(tgt_key_padding_mask[:,:tgt_len])
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len),dtype = torch.float32)).to(device)
        encoder_output = self.encoder(src=self.global_embedding(src), src_key_padding_mask = src_key_padding_mask)
#         print(encoder_output)
        decoder_output = self.decoder(tgt=self.global_embedding(tgt), memory=encoder_output, tgt_mask = tgt_mask,tgt_key_padding_mask=tgt_key_padding_mask[:,:tgt_len])
        output = self.output_linear(decoder_output)
        return output
    
    def decode(self,src,src_key_padding_mask,tgt,max_length = 512):
        with torch.no_grad():
            encoder_output = self.encoder(src=self.global_embedding(src), src_key_padding_mask = src_key_padding_mask)
        generation = 0
        while(generation<max_length):
            tgt_len = tgt.shape[1]
            tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len),dtype = torch.float32)).to(device) 
            with torch.no_grad():
#                 print(tgt)
                decoder_output = self.decoder(tgt=self.global_embedding(tgt), memory=encoder_output, tgt_mask = tgt_mask,tgt_key_padding_mask=None)
                output = self.output_linear(decoder_output)
                predicted_tokens = torch.argmax(self.soft(output)[:,-1,:], dim=-1).unsqueeze(1)
                tgt = torch.cat([tgt,predicted_tokens],dim=1)
            generation=generation+1
        return tgt
    

def get_preds(pred,target):

    avg_pred = 0
    F1 = 0
    for k in range(len(pred)):
        TP = 0
        FP = 0
        FN = 0
        pred_idx = pred[k][2:]
        avg_pred = avg_pred+len(pred_idx)-1
        target_i=target[k][2:]

        for i in pred_idx:
            if i not in [43620,43621,43622,43623]:
                if i in target_i:
                    TP = TP+1
                else:
                    FP = FP+1

        for i in target_i:
            if i not in pred_idx and i not in [43620,43621,43622,43623]:
                FN = FN+1
    
        F1 = F1 + TP/(TP+0.5*(FP+FN)+1e-10)
    
    F1 = F1/len(pred)
    avg_pred = avg_pred/len(pred)

    return F1,avg_pred
                    

In [ ]:
vocab_size = 43624 
d_model = 1024
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

model = EncoderDecoderTransformer(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers)
model.load_state_dict(torch.load("/kaggle/input/swapno-rec-encoder-decoder/swapno_next_bus.pth"))
# torch.save(model.state_dict(), "swapno_next_bus.pth")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.AdamW(model.parameters(), lr=0.00001, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, T_max=500, eta_min=0.00005)

In [ ]:
epochs = 100
steps = 0
for _ in range(epochs):
    losses = AverageMeter()
    F1s = AverageMeter()
    Lens = AverageMeter()
    
#     model.train()
    iterable = enumerate(train_loader)
    progress = None
    
    progress = tqdm(iterable, desc='Train', total=len(train_loader))
    iterable = progress
        
    for i, data in iterable:
        src_sequence = data["prev_prods"].to(device)
        tgt_sequence = data["next_basket"].to(device)
        src_key_padding_mask = data["src_mask"].to(device)
        tgt_key_padding_mask = data["tgt_mask"].to(device)
        
        optimiser.zero_grad()
        output = model(src_sequence,src_key_padding_mask, tgt_sequence[:,:-1],tgt_key_padding_mask)  # Exclude the last timestep from target
        loss = criterion(output[:,1:].flatten(0,1), tgt_sequence[:, 2:].flatten())  # Shift target by one timestep
        loss.backward()
        optimiser.step()
        scheduler.step()
        
        output_list = torch.argmax(model.soft(output[:,1:]),dim=-1).cpu().numpy().tolist()
        tgt_list = tgt_sequence[:, 2:].cpu().numpy().tolist()
        
        F1,Len = get_preds(output_list,tgt_list)
        F1s.update(F1, len(output_list))
        Lens.update(Len, len(output_list))
        
        if steps%500:
            wandb.log({"loss": losses.avg,
                       "F1": F1s.avg,
                       "Len":Lens.avg,
                       "lr": optimiser.param_groups[0]['lr'],
                       "epoch": steps})
        steps=steps+1
        
        losses.update(loss.detach().cpu(), tgt_key_padding_mask.shape[0])

        if progress is not None:
            progress.set_postfix_str('Loss:{ls:0.8f},F1:{f1:0.8f},Len:{ln:0.8f}'.format(
                ls=losses.avg,
                f1 = F1s.avg*100,
                ln = Lens.avg
            ))
#         break 
                      
    torch.save(model.state_dict(), "swapno_next_bus.pth")
#     break
    



In [ ]:
print(output_list)
print(" ")
print(tgt_list)

In [ ]:
# model.eval()
def decode(model,src,src_key_padding_mask,tgt,max_length = 512):
    soft = nn.Softmax(dim=-1)
    with torch.no_grad():
        encoder_output = model.encoder(src=model.global_embedding(src), src_key_padding_mask = src_key_padding_mask)
    generation = 0
#     print(model.global_embedding(src))
    print(encoder_output)
    while(generation<max_length):
        tgt_len = tgt.shape[1]
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len),dtype = torch.float32)).to(device) 
        tgt_key_padding_mask = torch.ones((tgt.shape[0], tgt_len),dtype=torch.float32).to(device)
#         print(tgt_mask)
        with torch.no_grad():
            
#             print(model.global_embedding(tgt).shape)
            decoder_output = model.decoder(tgt=model.global_embedding(tgt), memory=encoder_output, tgt_mask = tgt_mask,tgt_key_padding_mask=tgt_key_padding_mask)
            output = model.output_linear(decoder_output)
#             print(output[:,-1,:])
#             print(torch.sum(soft(output[:,-1,:]),dim=-1))
            predicted_tokens = torch.argmax(soft(output[:,-1,:]), dim=-1).unsqueeze(1)
#             print(predicted_tokens)
            tgt = torch.cat([tgt,predicted_tokens],dim=1)
#             print(tgt)
        generation=generation+1
#         break
    return tgt

start_tok = tgt_sequence[:,:2]
# print(start_tok)
# print(src_sequence)
y=decode(model,src_sequence,src_key_padding_mask,start_tok,50).cpu().numpy().tolist()
y = [i[:2]+list(set(i[2:])) for i in y]
print(y)

In [ ]:
model.train()
F1s = AverageMeter()
Lens = AverageMeter()

iterable = enumerate(check_loader)
progress = None

progress = tqdm(iterable, desc='Test', total=len(check_loader))
iterable = progress

for i, data in iterable:
    src_sequence = data["prev_prods"].to(device)
    tgt_sequence = data["next_basket"].to(device)
    src_key_padding_mask = data["src_mask"].to(device)
    tgt_key_padding_mask = data["tgt_mask"].to(device)
    start_tok = tgt_sequence[:,:2]
    decoded_outputs = model.decode(src_sequence,src_key_padding_mask,start_tok,50).cpu().numpy().tolist()
    decoded_outputs = [i[:2]+list(set(i[2:])) for i in decoded_outputs]
    tgt_sequence = tgt_sequence.cpu().numpy().tolist()
    tgt_sequence = [i[:2]+list(set(i[2:])) for i in tgt_sequence]
    F1,Len = get_preds(decoded_outputs,tgt_sequence)
    
    F1s.update(F1, len(decoded_outputs))
    Lens.update(Len, len(decoded_outputs))

    if progress is not None:
        progress.set_postfix_str('F1:{f1:0.8f}, Len:{le:6.2f}'.format(
            f1=F1s.avg*100,
            le=Lens.avg
        ))
    
    
    

In [ ]:
def decode(model,src,src_key_padding_mask,tgt,max_length = 512):
    with torch.no_grad():
        encoder_output = model.encoder(src=model.global_embedding(src), src_key_padding_mask = src_key_padding_mask)
    generation = 0
    while(generation<max_length):
        tgt_len = tgt.shape[1]
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len),dtype = torch.float32)).to(device) 
        with torch.no_grad():
            decoder_output = model.decoder(tgt=model.global_embedding(tgt), memory=encoder_output, tgt_mask = tgt_mask,tgt_key_padding_mask=None)
            output = model.output_linear(decoder_output)
            predicted_tokens = torch.argmax(model.soft(output)[:,-1,:], dim=-1).unsqueeze(1)
            tgt = torch.cat([tgt,predicted_tokens],dim=1)
        generation=generation+1
    return tgt

In [ ]:
model.train()

for data in check_loader:
    src_sequence = data["prev_prods"].to(device)
    tgt_sequence = data["next_basket"].to(device)
    src_key_padding_mask = data["src_mask"].to(device)
    tgt_key_padding_mask = data["tgt_mask"].to(device)
    start_tok = tgt_sequence[:,:2]
    decoded_outputs = model.decode(src_sequence,src_key_padding_mask,start_tok,50).cpu().numpy().tolist()
#     print(decoded_outputs)
    decoded_outputs = [i[:2]+list(set(i[2:])) for i in decoded_outputs]
    tgt_sequence = tgt_sequence.cpu().numpy().tolist()
    tgt_sequence = [i[:2]+list(set(i[2:])) for i in tgt_sequence]
    F1,Len = get_preds(decoded_outputs,tgt_sequence)
    
    for i in range(len(decoded_outputs)):
        print("Predicted:\n")
        for dp in decoded_outputs[i]:
            if dp not in [43620,43621,43622,43623]:
                print(global_code2name[global_idx2key[dp]])
        print("\nActual:\n")
        
        for tp in tgt_sequence[i]:
            if tp not in [43620,43621,43622,43623]:
                print(global_code2name[global_idx2key[tp]])
        print("\n\n")   
        break
    break
        
    
    

In [ ]:
torch.tensor(np.array([[7,8],[-8,9],[0,-5]]))

In [ ]:
soft = nn.Softmax(dim = -1)
y=soft(torch.tensor(np.array([[7,8],[-8,9],[0,-5]]).astype(np.float32)))
print(y)
torch.argmax(y,dim=-1)

In [ ]:
model.train()
def decode(model,src,src_key_padding_mask,tgt,max_length = 512):
    soft = nn.Softmax(dim=-1)
    with torch.no_grad():
        encoder_output = model.encoder(src=model.global_embedding(src), src_key_padding_mask = src_key_padding_mask)
    generation = 0
#     print(model.global_embedding(src))
#     print(encoder_output)
    while(generation<max_length):
        tgt_len = tgt.shape[1]
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len),dtype = torch.float32)).to(device) 
#         print(tgt_mask)
        with torch.no_grad():
#             print(model.global_embedding(tgt).shape)
            decoder_output = model.decoder(tgt=model.global_embedding(tgt), memory=encoder_output, tgt_mask = tgt_mask,tgt_key_padding_mask=None)
            output = model.output_linear(decoder_output)
#             print(output[:,-1,:])
#             print(torch.sum(soft(output[:,-1,:]),dim=-1))
            predicted_tokens = torch.argmax(soft(output[:,-1,:]), dim=-1).unsqueeze(1)
#             print(predicted_tokens)
            tgt = torch.cat([tgt,predicted_tokens],dim=1)
#             print(tgt)
        generation=generation+1
#         break
    return tgt

start_tok = tgt_sequence[:,:2]
# print(start_tok)
# print(src_sequence)
y=decode(model,src_sequence,src_key_padding_mask,start_tok,50).cpu().numpy().tolist()

In [ ]:
print(y)

In [ ]:
# torch.ones((10,1))